# Unpivot Other Columns
> shouldn't this be easier?

- toc: true 
- badges: true
- comments: true
- categories: [jupyter, spark, pyspark, databricks]
- hide: false

In [0]:
import pandas as pd

data = {
  "continent": ["Americas", "Americas", "Asia", "Europe"],
  "country": [ "USA", "Mexico", "Japan", "France"],
  "Population 2014": [1, 2, 3, 4],
  "Population 2015": [5, 6, 7, 8],
}

df = pd.DataFrame(data)
df

,continent,country,Population 2014,Population 2015
0,Americas,USA,1,5
1,Americas,Mexico,2,6
2,Asia,Japan,3,7
3,Europe,France,4,8


In [0]:
df.melt(id_vars=["continent", "country"])

,continent,country,variable,value
0,Americas,USA,Population 2014,1
1,Americas,Mexico,Population 2014,2
2,Asia,Japan,Population 2014,3
3,Europe,France,Population 2014,4
4,Americas,USA,Population 2015,5
5,Americas,Mexico,Population 2015,6
6,Asia,Japan,Population 2015,7
7,Europe,France,Population 2015,8


In [0]:
spark_df = spark.createDataFrame(df)
display(spark_df)

continent,country,Population 2014,Population 2015
Americas,USA,1,5
Americas,Mexico,2,6
Asia,Japan,3,7
Europe,France,4,8


In [0]:
from pyspark.sql import DataFrame
from typing import Iterable 

def unpivot_df(
        df: DataFrame, 
        id_cols: Iterable[str], pvt_cols: Iterable[str]=None,
        col_name: str="Column", val_name: str="Value") -> DataFrame:

  _df = df
  _cols = set(_df.columns)
  _id_cols = id_cols

  if pvt_cols is None:
    _pvt_cols = list(_cols.difference(id_cols))
  else:
    _pvt_cols = pvt_cols

  _stack_expr = ', '.join(["'{}', `{}`".format(_c, _c) for _c in _pvt_cols])
  _stack_expr = f"""stack({len(_pvt_cols)}, {_stack_expr}) as ({col_name}, {val_name})"""
  _unpvt_expr = _id_cols + [_stack_expr, ]
  _unpvt_df = _df.selectExpr(*_unpvt_expr)
  
  return _unpvt_df

In [0]:
spark_df_long = unpivot_df(spark_df, ["continent", "country"])
display(spark_df_long)

continent,country,Column,Value
Americas,USA,Population 2015,5
Americas,USA,Population 2014,1
Americas,Mexico,Population 2015,6
Americas,Mexico,Population 2014,2
Asia,Japan,Population 2015,7
Asia,Japan,Population 2014,3
Europe,France,Population 2015,8
Europe,France,Population 2014,4


> et voila